In [ ]:
# ============================================================
# Flood Disaster Simulation Backend (FINAL + INPUT VALIDATION)
# Flask + ngrok | Frontend-Compatible | PPT-Compliant
# ============================================================

!pip install -q flask flask-cors pillow requests numpy opencv-python-headless torch torchvision diffusers transformers accelerate pyngrok scikit-image perlin-noise timm > /dev/null || true

# -------------------------
# 1) Imports
# -------------------------
import os, io, json, time, math, traceback
from flask import Flask, request, send_file, make_response, jsonify
from flask_cors import CORS
from PIL import Image, ImageDraw
import numpy as np
import cv2
import requests
from pyngrok import ngrok
from skimage.metrics import structural_similarity as ssim
from scipy.linalg import sqrtm
from perlin_noise import PerlinNoise
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from transformers import (
    CLIPProcessor, CLIPModel,
    AutoImageProcessor, AutoModelForSemanticSegmentation,
    BlipProcessor, BlipForConditionalGeneration,
    AutoTokenizer, AutoModelForCausalLM
)
from diffusers import StableDiffusionInpaintPipeline

# ============================================================
# 2) CONFIGURATION 
# ============================================================

HF_TOKEN = "YOUR HUGGING FACE TOKEN"
NGROK_TOKEN = "YOUR NGROK_TOKEN"

# ---- Google Earth Engine (Conceptual) ----
GEE_PROJECT_ID = "GOOGLE PROJECT_ID"
GEE_DATASET = "COPERNICUS/S2"

# ---- ChatGPT / LLM (Conceptual) ----
CHATGPT_MODEL_NAME = "GPT-4 / GPT-3.5 (conceptual)"

# ============================================================
# 3) App Setup
# ============================================================

app = Flask(__name__)
CORS(app)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# ============================================================
# 4) Load Active Models
# ============================================================

# ---- CLIP (USED for validation) ----
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model.eval()

# ---- SegFormer ----
seg_processor = AutoImageProcessor.from_pretrained("nvidia/segformer-b4-finetuned-ade-512-512")
seg_model = AutoModelForSemanticSegmentation.from_pretrained(
    "nvidia/segformer-b4-finetuned-ade-512-512"
).to(device)
seg_model.eval()

# ============================================================
# 5)  MODELS 
# ============================================================

# ---- BLIP ----
try:
    blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    blip_model = BlipForConditionalGeneration.from_pretrained(
        "Salesforce/blip-image-captioning-base"
    ).to(device)
    blip_model.eval()
except Exception:
    blip_model = None
    blip_processor = None

# ---- GPT (local placeholder) ----
try:
    gpt_tokenizer = AutoTokenizer.from_pretrained("gpt2")
    gpt_model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)
    gpt_model.eval()
except Exception:
    gpt_model = None
    gpt_tokenizer = None

# ---- Stable Diffusion (optional) ----
sd_pipe = None
try:
    sd_pipe = StableDiffusionInpaintPipeline.from_pretrained(
        "runwayml/stable-diffusion-inpainting",
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        use_auth_token=HF_TOKEN
    ).to(device)
    sd_pipe.safety_checker = None
except Exception:
    sd_pipe = None

# ============================================================
# 6) PLACEHOLDER FUNCTIONS 
# ============================================================

def fetch_image_from_gee(lat, lon):
    """
    Placeholder for Google Earth Engine Sentinel-2 imagery.
    Not invoked due to authentication constraints.
    """
    return None

def generate_prompt_with_blip_and_gpt(pil_img):
    """
    BLIP + GPT conceptual pipeline (not used).
    """
    return {"caption": "urban area", "prompt": "urban area affected by flooding"}

def enhance_prompt_with_chatgpt(prompt):
    """
    ChatGPT conceptual enhancement.
    """
    return prompt + " with post-disaster flood damage"

# ============================================================
# 7) TILE FETCHING
# ============================================================

def latlon_to_tile(lat, lon, zoom):
    lat_rad = math.radians(lat)
    n = 2.0 ** zoom
    return int((lon + 180) / 360 * n), int(
        (1 - math.log(math.tan(lat_rad) + 1 / math.cos(lat_rad)) / math.pi) / 2 * n
    )

def fetch_512_tile(lat, lon, zoom=16):
    cx, cy = latlon_to_tile(lat, lon, zoom)
    tiles = []
    for dy in [0,1]:
        for dx in [0,1]:
            r = requests.get(
                f"https://mt1.google.com/vt/lyrs=s&x={cx+dx}&y={cy+dy}&z={zoom}",
                timeout=20
            )
            tiles.append(Image.open(io.BytesIO(r.content)).convert("RGB"))
    img = Image.new("RGB", (512,512))
    img.paste(tiles[0], (0,0)); img.paste(tiles[1], (256,0))
    img.paste(tiles[2], (0,256)); img.paste(tiles[3], (256,256))
    return img

# ============================================================
# 8) VALIDATION + CORE PROCESSING
# ============================================================

def is_satellite_image(pil_img):
    """
    CLIP-based validation to reject human / street-level images.
    """
    prompts = [
        "a satellite image",
        "an aerial view",
        "a top-down satellite photograph"
    ]
    negative = [
        "a portrait photo",
        "a human face",
        "a street-level photo",
        "a selfie"
    ]
    inputs = clip_processor(
        text=prompts + negative,
        images=pil_img,
        return_tensors="pt",
        padding=True
    ).to(device)

    with torch.no_grad():
        logits = clip_model(**inputs).logits_per_image.softmax(dim=1)[0]

    pos_score = logits[:len(prompts)].mean().item()
    neg_score = logits[len(prompts):].mean().item()
    return pos_score > neg_score + 0.03

def compute_water_mask_rgb(rgb_pil):
    arr = np.array(rgb_pil).astype(np.float32)
    ndwi = (arr[:,:,1] - arr[:,:,2]) / (arr[:,:,1] + arr[:,:,2] + 1e-8)
    return (ndwi > 0.03).astype(np.uint8) * 255

def generate_flood_using_noise(pil_img):
    mask = compute_water_mask_rgb(pil_img)
    arr = np.array(pil_img)
    arr[mask > 0] = (arr[mask > 0]*0.55 + np.array([80,120,200])*0.45).astype(np.uint8)
    return Image.fromarray(arr), mask

# ============================================================
# 9) HOTSPOT DETECTION
# ============================================================

def detect_hotspots(mask):
    contours,_ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    hotspots=[]
    for c in contours:
        area = int(cv2.contourArea(c))
        if area < 60:
            continue
        M = cv2.moments(c)
        if M["m00"] == 0:
            continue
        cx = int(M["m10"]/M["m00"])
        cy = int(M["m01"]/M["m00"])
        sev = "high" if area > 4000 else "medium" if area > 1500 else "low"
        hotspots.append({
            "x": cx,
            "y": cy,
            "area": area,
            "severity": sev
        })
    return hotspots

# ============================================================
# 10) METRICS
# ============================================================

_inception = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)
_inception.fc = torch.nn.Identity()
_inception.eval()

def compute_fid(a,b):
    tf = transforms.Compose([transforms.Resize((299,299)), transforms.ToTensor()])
    fa = _inception(tf(a).unsqueeze(0)).detach().numpy()
    fb = _inception(tf(b).unsqueeze(0)).detach().numpy()
    return float(np.sum((fa.mean(0)-fb.mean(0))**2))

# ============================================================
# 11) ROUTES (Frontend-Compatible)
# ============================================================

@app.route("/fetch_before")
def fetch_before():
    lat = float(request.args["lat"])
    lon = float(request.args["lon"])
    img = fetch_512_tile(lat, lon)
    buf = io.BytesIO(); img.save(buf,"PNG"); buf.seek(0)
    return send_file(buf, mimetype="image/png")

@app.route("/simulate", methods=["POST"])
def simulate():
    try:
        if "file" in request.files:
            before = Image.open(request.files["file"]).convert("RGB").resize((512,512))

            # 🚫 INPUT VALIDATION
            if not is_satellite_image(before):
                return jsonify({
                    "error": "Invalid input image. Please upload a satellite or aerial image."
                }), 400
        else:
            before = fetch_512_tile(
                float(request.form["lat"]),
                float(request.form["lon"])
            )

        after, mask = generate_flood_using_noise(before)
        hotspots = detect_hotspots(mask)

        ssim_v = ssim(
            np.array(before.convert("L")),
            np.array(after.convert("L"))
        )
        fid_v = compute_fid(before, after)
        flood_pct = round(mask.mean()/255*100,2)

        metrics = {
            "ssim": round(ssim_v,3),
            "fid": round(fid_v,3),
            "flood_percent": flood_pct
        }

        buf = io.BytesIO()
        after.save(buf,"PNG")
        buf.seek(0)

        resp = make_response(send_file(buf, mimetype="image/png"))
        resp.headers["X-Metrics"] = json.dumps(metrics)
        resp.headers["X-Hotspots"] = json.dumps(hotspots)
        resp.headers["X-Workflow-Type"] = "SEG_NOISE"
        return resp

    except Exception as e:
        traceback.print_exc()
        return jsonify({"error": str(e)}), 500

@app.route("/compare", methods=["POST"])
def compare():
    try:
        gen = Image.open(request.files["generated"]).convert("RGB").resize((512,512))
        real = Image.open(request.files["real"]).convert("RGB").resize((512,512))

        mask_gen = compute_water_mask_rgb(gen)
        mask_real = compute_water_mask_rgb(real)

        TP = int(np.sum((mask_gen>0)&(mask_real>0)))
        FP = int(np.sum((mask_gen>0)&(mask_real==0)))
        FN = int(np.sum((mask_gen==0)&(mask_real>0)))
        TN = int(np.sum((mask_gen==0)&(mask_real==0)))

        accuracy = (TP+TN)/(TP+TN+FP+FN+1e-9)*100
        precision = TP/(TP+FP+1e-9)
        recall = TP/(TP+FN+1e-9)
        f1 = 2*precision*recall/(precision+recall+1e-9)

        ssim_v = ssim(
            np.array(real.convert("L")),
            np.array(gen.convert("L"))
        )
        fid_v = compute_fid(real, gen)

        return jsonify({
            "accuracy": round(accuracy,3),
            "precision": round(precision,3),
            "recall": round(recall,3),
            "f1": round(f1,3),
            "ssim": round(ssim_v,3),
            "fid": round(fid_v,3)
        })

    except Exception as e:
        traceback.print_exc()
        return jsonify({"error": str(e)}), 500

# ============================================================
# 12) START SERVER
# ============================================================

ngrok.set_auth_token(NGROK_TOKEN)
public_url = ngrok.connect(5000).public_url
print("🚀 BACKEND URL:", public_url)

app.run(host="0.0.0.0", port=5000, use_reloader=False)
